In [2]:
import os
os.environ["KERAS_BACKEND"] = "jax" # you can also use tensorflow or torch

import keras
from keras import ops
import tensorflow as tf

import cv2
import pandas as pd
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
import joblib

import matplotlib.pyplot as plt 

2024-04-04 20:55:56.335881: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-04 20:55:56.335958: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-04 20:55:56.337171: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-04 20:55:56.344537: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
%matplotlib inline
%pip install --quiet torchvision

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

In [ ]:
BASE_PATH = "/blue/esi4611/share/planttraits2024"

In [ ]:
#Configuration
class CFG:
    verbose = 1  # Verbosity
    seed = 42  # Random seed
    preset = "efficientnetv2_b2_imagenet"  # Name of pretrained classifier
    image_size = [224, 224]  # Input image size
    epochs = 12 # Training epochs
    batch_size = 96  # Batch size
    lr_mode = "step" # LR scheduler mode from one of "cos", "step", "exp"
    drop_remainder = True  # Drop incomplete batches
    num_classes = 6 # Number of classes in the dataset
    num_folds = 5 # Number of folds to split the dataset
    fold = 0 # Which fold to set as validation data
    class_names = ['X4_mean', 'X11_mean', 'X18_mean',
                   'X26_mean', 'X50_mean', 'X3112_mean',]
    aux_class_names = list(map(lambda x: x.replace("mean","sd"), class_names))
    num_classes = len(class_names)
    aux_num_classes = len(aux_class_names)

In [ ]:
# Train + Valid
train_data = pd.read_csv(f'{BASE_PATH}/train.csv')
train_data['image_path'] = f'{BASE_PATH}/train_images/'+train_data['id'].astype(str)+'.jpeg'
train_data.loc[:, CFG.aux_class_names] = train_data.loc[:, CFG.aux_class_names].fillna(-1)
display(train_data.head(2))

# Test
test_data = pd.read_csv(f'{BASE_PATH}/test.csv')
test_data['image_path'] = f'{BASE_PATH}/test_images/'+test_data['id'].astype(str)+'.jpeg'
FEATURE_COLS = test_data.columns[1:-1].tolist()
display(test_data.head(2))

# 📩 | Submission

In [23]:
pred_df = test_data[["id"]].copy()
target_cols = [x.replace("_mean","") for x in CFG.class_names]
pred_df[target_cols] = preds.tolist()

sub_df = pd.read_csv(f'{BASE_PATH}/sample_submission.csv')
sub_df = sub_df[["id"]].copy()
sub_df = sub_df.merge(pred_df, on="id", how="left")
sub_df.to_csv("submission.csv", index=False)
sub_df.head()

,id,X4,X11,X18,X26,X50,X3112
0,195066138,-0.099060,1.404266,0.069888,1.254300,-0.025760,121.394836
1,195524180,0.220290,2.242062,2.832736,9.946978,0.370434,151.271317
2,172220512,-0.269862,0.496115,0.083798,4.467178,-0.191250,149.784225
3,195824490,-0.455548,-0.053600,-0.515895,-0.141736,-0.425166,43.501873
4,191793047,-0.314372,0.328862,-0.089717,0.208312,-0.252556,95.088547
